In [2]:
import pandas as pd
import numpy as np

# Опыт с MultiLabelBinarizer

**Создадим набор данных**

In [3]:
d = pd.DataFrame({'text': ['Сладкий чай', 'Милое лицо', 'Сладкий чай'], 
              't': ['Чай', 'Лицо', 'Чай'], 
              'p': ['Чай', 'Лицо', 'Ягода']})
d.head()

,text,t,p
0,Сладкий чай,Чай,Чай
1,Милое лицо,Лицо,Лицо
2,Сладкий чай,Чай,Ягода


In [4]:
agr = d.groupby(by='text', group_keys=False).agg(t=('t','unique'), p=('p','unique'))
agr.head()

,t,p
text,,
Милое лицо,[Лицо],[Лицо]
Сладкий чай,[Чай],"[Чай, Ягода]"


In [5]:
agr = d.groupby(by='text').agg(['unique'])
agr.columns = agr.columns.droplevel(1)
agr.head()

,t,p
text,,
Милое лицо,[Лицо],[Лицо]
Сладкий чай,[Чай],"[Чай, Ягода]"


In [6]:
d[['t', 'p']].agg('unique')

array(['Лицо', 'Чай', 'Ягода'], dtype=object)

In [7]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import precision_score, recall_score

mlb = MultiLabelBinarizer(classes=d[['t', 'p']].agg('unique'))

gold_labels = mlb.fit_transform(agr['t'])
pred_labels = mlb.transform(agr['p'])

recall_score(gold_labels, pred_labels, average='samples')

1.0

# Подсчёт метрик

In [69]:
train = pd.read_csv('data/processed/perfumery_train.csv').sort_values('frequency', ascending=False)[['phrase', 'subtopic']]
train.head()

,phrase,subtopic,frequency,lemmas
0,флакон,Флакон,88474,флакон
1,шлейф,След аромата,51317,шлейф
2,роза,Цветочные,35609,роза
3,раскрывается,Время раскрытия,33018,раскрываться
4,нота,Нота,31010,нота


In [261]:
all_metrics = pd.read_csv('data/model/0.85_500_all_metrics.csv')
all_metrics.tail()

,f1,precision,recall,validation_size,model_from_val,model_from_all,people_from_val
38,0.926896,0.905446,0.986315,19657,19235,0,19500
39,0.925339,0.903805,0.985811,20157,19778,0,20000
40,0.925578,0.903919,0.985380,20657,20275,0,20500
41,0.924749,0.902891,0.984970,21157,20727,0,21000
42,0.927303,0.905535,0.988445,21550,21226,0,21393


In [262]:
m = all_metrics[-3:][['f1', 'precision', 'recall']].agg('mean')
m

f1           0.925877
precision    0.904115
recall       0.986265
dtype: float64

In [263]:
all_metrics.iloc[-1:, :3]

,f1,precision,recall
42,0.927303,0.905535,0.988445


In [272]:
all_metrics.iloc[-1:, :3] = all_metrics.iloc[-3:, :3].agg('mean')

In [267]:
all_metrics[-3:][['f1', 'precision', 'recall']].agg('mean')

f1           0.925402
precision    0.903642
recall       0.985538
dtype: float64

In [293]:
x = [[1], [2], [1, 2]]

In [298]:
np.array(x, dtype='object')[[1, 2]]

array([list([2]), list([1, 2])], dtype=object)

# Отладка ошибки по Explode

In [40]:
test = pd.read_csv('testt.csv')
test.head()

,Unnamed: 0,phrase,true,subtopic
0,0,проходной парфюм,Продукт,Продукт
1,0,проходной парфюм,Продукт,Мужской
2,1,процентное содержание,Список ингредиентов,Список ингредиентов
3,1,процентное содержание,Список ингредиентов,Алкоголя/спирта
4,1,процентное содержание,Список ингредиентов,Количество/объем


**Загрузка данных**

In [41]:
t = test.groupby(by='phrase').agg(subtopic=('subtopic', 'unique'), true=('true', 'unique'))
t

,subtopic,true
phrase,,
проходной парфюм,"[Продукт, Мужской]",[Продукт]
процентное содержание,"[Список ингредиентов, Алкоголя/спирта, Количес...",[Список ингредиентов]
процентное соотношение,"[Сочетание, Список ингредиентов, Количество/об...",[Список ингредиентов]
процентом алкоголя,"[Нота, Оттенок, Алкоголя/спирта]",[Список ингредиентов]
процентом содержания,"[Список ингредиентов, Количество/объем, Алкого...",[Список ингредиентов]
...,...,...
пудринка,"[Кислый, Пирамида, След аромата, Острый, Ягодный]",[Пудровый]
пудринкой,"[Специи, Кислый, Цветочные, Горький, Цитрусовые]",[Пудровый]
пудровая,"[Пудровый, Нежный, Ореха/ореховый]",[Пудровый]


In [42]:
labels = np.array([], dtype='object')
for i in range(len(t)):
    labels = np.append(labels, [*t['subtopic'][i], *t['true'][i]])
labels = np.unique(labels)

In [13]:
t['subtopic'][0] = np.append(t['subtopic'][0], ['Тропический аромат'])
t

,subtopic,true
phrase,,
avonназвание,"[Ореха/ореховый, Звучание, Полноразмерное сред...",[Название бренда/автора]
avonстоимость,"[Амбра/Амбровый, Стойкость, Удовые, Количество...",[Название бренда/автора]
brocardколлекция,"[Палитра, Миниатюра, Линейка, Серия, Композиция]",[Название бренда/автора]
cвежий,"[Амбра/Амбровый, Ореха/ореховый, Сладкий, Свеж...",[Свежий]
cладкий,"[Амбра/Амбровый, Ореха/ореховый, Сладкий, Остр...",[Сладкий]


In [155]:
t['true'][4] = np.append(t['true'][4], ['Цитрусовые'])
t

,subtopic,true
phrase,,
аромат альдегидов,"[Альдегида, Аромат листьев, Аромат смолы, Аром...","[Альдегида, Тропический аромат, Аромат листьев]"
аромат ананаса,"[Тропический аромат, Алкоголя/спирта, Аромат л...",[Тропический аромат]
аромат ангелов,"[Название бренда/автора, Алкоголя/спирта, Альд...",[Название аромата]
аромат апельсина,"[Цитрусовые, Тропический аромат, Аромат листье...",[Цитрусовые]
аромат апельсинового цвета,"[Цветочные, Цитрусовые, Шампанского, Аромат ли...","[Цвет, Цветочные, Шампанского, Цитрусовые]"


In [43]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import precision_score, recall_score, f1_score

mlb = MultiLabelBinarizer(classes=labels)

gold_labels = mlb.fit_transform(t['true'].values)
pred_labels = mlb.transform(t['subtopic'].values)

precision_score(y_true=pred_labels, y_pred=gold_labels, average='samples')

0.44

In [44]:
recall_score(y_true=pred_labels, y_pred=gold_labels, average='samples')

0.1156

In [45]:
f1_score(pred_labels, gold_labels, average='samples')

0.17600000000000002

**Количество вхождений**

In [46]:
count = 0
for i in range(t.shape[0]):
    if t['true'].values[i][0] in t['subtopic'].values[i]:
        count += 1
count

<h1 align='center'>leave one out</h1>

In [2]:
import pandas as pd

In [30]:
metrics = pd.read_csv('data/model/0.9_all_metrics.csv')
metrics.head()

,f1,precision,recall,validation_size,phrase
0,0.0,0.0,0.00,1,avonназвание
1,0.0,0.0,0.00,1,avonстоимость
2,0.0,0.0,0.00,1,brocardколлекция
3,0.0,0.0,0.00,1,cвежий
4,0.4,1.0,0.25,1,cладкий


In [13]:
predicts = pd.read_csv('data/model/0.9_predicts.csv')
predicts.head()

,phrase,subtopic,true
0,avonназвание,['Серия' 'Линейка' 'Дизайн упаковки' 'Название...,Название бренда/автора
1,avonстоимость,['Страна производства' 'Люкс' 'Продукт' 'Бюдже...,Название бренда/автора
2,brocardколлекция,['Женский' 'Страна производства' 'Продукт' 'Ни...,Название бренда/автора
3,cвежий,['Нежный' 'Сладкий' 'Цветочные' 'Острый' 'Цитр...,Свежий
4,cладкий,['Свежий' 'Сладкий' 'Флакон' 'Острый'],Сладкий


In [28]:
predicts['subtopic'][0].replace('[', '').replace(']', '').split("'")

['',
 'Серия',
 ' ',
 'Линейка',
 ' ',
 'Дизайн упаковки',
 ' ',
 'Название аромата',
 '']

In [31]:
metrics.agg('mean')

C:\Users\bimas\AppData\Local\Temp\ipykernel_5540\794148342.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  metrics.agg('mean')


f1                 0.465500
precision          0.915557
recall             0.349431
validation_size    1.000000
dtype: float64

In [38]:
metrics.loc[metrics['precision'] == 0]

,f1,precision,recall,validation_size,phrase
0,0.0,0.0,0.0,1,avonназвание
1,0.0,0.0,0.0,1,avonстоимость
2,0.0,0.0,0.0,1,brocardколлекция
3,0.0,0.0,0.0,1,cвежий
8,0.0,0.0,0.0,1,faberlicгод
...,...,...,...,...,...
11428,0.0,0.0,0.0,1,попроще аккорд
11443,0.0,0.0,0.0,1,попроще прямоугольный флакон
11445,0.0,0.0,0.0,1,попроще спрей
11446,0.0,0.0,0.0,1,попроще стеклянный флакон


# perfumery

In [14]:
m = pd.read_csv('data/model/0.9_250_all_metrics.csv')
m['precision'].mean()

0.7254409127318356

In [15]:
m = pd.read_csv('data/model/0.9_250_marked_metrics.csv')
m['precision'].mean()

0.4267359433701856

# best

In [16]:
m = pd.read_csv('data/model/0.9_250_all_metrics.csv')
m['precision'].mean()

0.7108103532188981

In [17]:
m = pd.read_csv('data/model/0.9_250_marked_metrics.csv')
m['precision'].mean()

0.39694323279239685

# new

In [18]:
m = pd.read_csv('data/model/0.9_250_all_metrics.csv')
m['precision'].mean()

0.7316168050919891

In [19]:
m = pd.read_csv('data/model/0.9_250_marked_metrics.csv')
m['precision'].mean()

0.47776793932646644

# cc300

In [23]:
m = pd.read_csv('data/model/0.9_250_all_metrics.csv')
m['precision'].mean()

0.6800219494755948

In [24]:
m = pd.read_csv('data/model/0.9_250_marked_metrics.csv')
m['precision'].mean()

0.6753190751290618